# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770528798123                   -0.53    9.0         
  2   -2.772138938518       -2.79       -1.29    1.0    200ms
  3   -2.772170231934       -4.50       -2.69    1.0    159ms
  4   -2.772170721496       -6.31       -3.97    2.0    224ms
  5   -2.772170722958       -8.83       -4.74    1.0    211ms
  6   -2.772170723011      -10.28       -5.51    1.0    166ms
  7   -2.772170723015      -11.43       -5.85    2.0    218ms
  8   -2.772170723015      -12.65       -7.01    1.0    174ms
  9   -2.772170723015      -14.65       -7.20    2.0    203ms
 10   -2.772170723015   +  -14.03       -7.29    1.0    199ms
 11   -2.772170723015      -14.40       -8.16    1.0    184ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770593581529                   -0.53    9.0         
  2   -2.7

1.7735579952481817

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770720669540                   -0.53    9.0         
  2   -2.772055739881       -2.87       -1.31    1.0    234ms
  3   -2.772083155689       -4.56       -2.55    1.0    158ms
  4   -2.772083385653       -6.64       -3.50    1.0    225ms
  5   -2.772083415879       -7.52       -3.90    2.0    185ms
  6   -2.772083417746       -8.73       -5.11    1.0    165ms
  7   -2.772083417810      -10.20       -5.47    2.0    224ms
  8   -2.772083417811      -11.98       -5.93    2.0    193ms
  9   -2.772083417811   +  -14.88       -6.59    1.0    203ms
 10   -2.772083417811   +  -14.21       -6.96    1.0    183ms
 11   -2.772083417811      -13.74       -8.09    1.0    174ms

Polarizability via ForwardDiff:       1.7725349701278148
Polarizability via finite difference: 1.7735579952481817
